<a href="https://colab.research.google.com/github/Deven10103/MachineLearning.explore/blob/main/LLMs/Pdf_Summariser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gradio openai pypdf tiktoken langchain langchain_community langchain-openai
# Installing the packages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 1.5 MB/s eta 0:00:00


In [ ]:
!pip install python-dotenv
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="/content/sys.env")

openai = os.getenv("OPENAI_API_KEY")

In [ ]:
import tiktoken

def tokenize(string: str,encodingnm: str) -> int:
  encoding = tiktoken.get_encoding(encodingnm)
  num_tokens=len(encoding.encode(string))
  print(encoding.encode(string))
  return num_tokens

tokenize("tiktoken is being used.","cl100k_base")

[83, 1609, 5963, 374, 1694, 1511, 13]


7

In [ ]:
import gradio
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader

llm=ChatOpenAI(temperature=0)

In [ ]:
def summarize(file_path):
  loader=PyPDFLoader(file_path)
  docs =loader.load_and_split()
  chain = load_summarize_chain(llm=llm, chain_type="map_reduce")
  summary=chain.run(docs)

  return summary

In [1]:
summarize("/content/HorchreiterSchmidhuber_LSTM.pdf")


Could not proceed.
